# Fine Tuting Gemma 2b on Custom Dataset

## Installing Dependencies

In [ ]:
# !pip3 install -q -U accelerate==0.27.1
!pip install -Uqqq
!pip install -qqq bitsandbytes accelerate
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.9 MB/s eta 0:00:00


## Importing Essential Libraries

In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer




## Accessing HuggingFace token for authentication

In [ ]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

## Creating Model and Quantization Configuration

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit =True,
    bnb_4bit_qaunt_type = "nf4",
    bnb_4bit_compute_dtypr = torch.bfloat16
)

## Initializing the tokenizer and LLM model using HuggingFace Pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token = os.environ["HF_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map = "auto",
                                             token = os.environ["HF_TOKEN"])

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = "Quote: Imagination is more, "
device = "cuda:0"
inputs = tokenizer(text, return_tensors = 'pt').to(device)

outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Quote: Imagination is more, 
Than thought and more than logic.
Imagination is the moving source of thought,
Imagination is the mother of all knowledge.
Imagination is the genesis of all art.
Imagination is the mother of all science.
Imagin


In [ ]:
  os.environ['WANDB_DISABLED'] = "false"

## Configuring the LoRA pipeline

In [ ]:
lora_config = LoraConfig(\
    r = 8,\
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",\
)

## Downloading Dataset required for finetuning

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']) , batched = True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
def formatting(data_sample):
  text = f"Quote: {data_sample['quote'][0]}\nAuthor: {data_sample['author'][0]}"
  return [text]

## Configuring the Training pipeline with model, dataset, data formating and finetuning configurations

In [ ]:
trainer = SFTTrainer(
                model = model,
                train_dataset = data['train'],
                args = transformers.TrainingArguments(per_device_train_batch_size = 1,
                                                        gradient_accumulation_steps = 4,
                                                        warmup_steps = 2,
                                                        max_steps = 1000,
                                                        learning_rate = 2e-4,
                                                        fp16 = True,
                                                        logging_steps = 1,
                                                        output_dir = "outputs",
                                                        optim = "paged_adamw_8bit"),
                peft_config = lora_config,
                formatting_func = formatting,
)

## Training model for finetuning and Testing

In [ ]:
## Model training
trainer.train()

In [ ]:
## Testing
test_text = "Quote: A woman is like a tea bag;"
device = "cuda:0"
inputs = tokenizer(test_text, return_tensors = 'pt').to(device)
outputs = model.generate(**inputs, max_new_tokens = 20)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

Quote: A woman is like a tea bag; you never know how strong it is ‘till it’s in hot water.
Author: Author
